In [1]:
import yfinance as yf  
import numpy as np
import pandas as pd

In [2]:
# Get a list of S&P 500 Ticker
S_P_500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
ticker_set = S_P_500['Symbol'].tolist()

**Scrap data from yfinance** <br>
Function 'parameter' in the last cell of this notebook. <br>
Please refer to here for the concepts <br>
https://github.com/lauwo711/New-Factor-inspired-by-Momentum/blob/master/New%20Factor%20inspired%20by%20Momentum.ipynb

In [ ]:
data_set = []
for i in ticker_set:
    try:
        print(i)
        data_set.append(parameter(i,'2010-01-01','2019-12-31'))
    except (ValueError,IndexError,ZeroDivisionError):
        pass

In [35]:
# display data
pd.DataFrame(np.array(data_set),columns=['Ticker','Return','num_up', 'num_down', 'speed_up', 'speed_down']).head()

,Ticker,Return,num_up,num_down,speed_up,speed_down
0,MMM,1.75491570924234,62,39,0.24102981351981342,-0.4434347002724796
1,ABT,3.7826775503853565,68,38,0.08953383472057072,-0.11256637997432599
2,ABBV,2.3026503638205766,48,39,0.18662609496124025,-0.2296618404907975
3,ABMD,18.162657502863688,74,63,0.6041620498614958,-0.6489597249508842
4,ACN,5.279990499726416,69,43,0.2076265734265734,-0.27755870653685666


In [15]:
# get data n years from today
def get_price(ticker,start_date,end_date):
    px = yf.download(ticker,start=start_date,end=end_date)['Adj Close'].to_numpy()
    return px[~np.isnan(px)]

# get n days moving average price
def n_days_moving_avg(px,n):
    ma = np.zeros((len(px)-n+1)) + px[n-1:] #initialize array ma
    for i in range(n-1):
        ma += px[n-2-i:-1-i]
    return ma/n

def parameter(ticker,start_date,end_date):
    px = np.round(get_price(ticker,start_date,end_date),4)
    ma_px = n_days_moving_avg(px,10)

    sec_diff = np.diff(np.sign(np.diff(ma_px)))

    # find out turning points
    turn_pt = []
    for i in range(len(sec_diff)):
        if sec_diff[i]<0:
            turn_pt.append([i, ma_px[1:-1][i]])
        elif sec_diff[i]>0:
            turn_pt.append([i, ma_px[1:-1][i]])

    # filter significant points using threshold=2%
    turn_pt = [turn_pt[i] for i in range(len(turn_pt)-1) if abs(turn_pt[i+1][1]/turn_pt[i][1] - 1) > 0.02]

    #speed
    turn_pt = np.array(turn_pt).transpose()
    dist = np.diff(turn_pt[1])
    time = np.diff(turn_pt[0])
    speed = dist/time

    r = px[-1]/px[0] - 1

    num_up = len([i for i in speed if i > 0])

    num_down = len([i for i in speed if i < 0])

    speed_up = sum([i for i in dist if i > 0]) / sum([time[i] for i in range(len(dist)) if dist[i] > 0])

    speed_down = sum([i for i in dist if i < 0]) / sum([time[i] for i in range(len(dist)) if dist[i] < 0])

    return [ticker, r, num_up, num_down, speed_up, speed_down]